<a href="https://colab.research.google.com/github/gfjf/gfjf.github.io/blob/main/C%C3%B3pia_de_ELE575_Semana4_115095.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Aprendizado Profundo
##Atividade Prática 3


Aplique a técnica de transfer learning para treinar um novo modelo para classificar o dataset CIFAR-10, usado na Atividade 2.

Comente sobre os resultados obtidos neste experimento quando comparados com os resultados obtidos na realização da Atividade 2.

Foi mais fácil e rápido treinar o modelo usando transfer learning?



###Carregamos as bibliotecas

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage
import os
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from sklearn.model_selection import KFold
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model, Sequential
from keras.layers import Lambda, Input
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator



###Carregamos o dataset

Usaremos o dataset CIFAR-10 (https://www.cs.toronto.edu/~kriz/cifar.htm), que consiste em 60.000 imagens coloridas de 32x32x3 (32x32 pixels, com 3 canais de cores RGB), distribuídas uniformemente em 10 classes. Existem 50.000 imagens de treinamento e 10.000 imagens de teste.

O conjunto de dados é dividido em cinco lotes de treinamento e um lote de teste, cada um com 10.000 imagens. O lote de teste contém exatamente 1.000 imagens selecionadas aleatoriamente de cada classe. Os lotes de treinamento contêm as imagens restantes em ordem aleatória, mas alguns lotes de treinamento podem conter mais de uma classe do que de outra. Entre eles, os lotes de treinamento contêm exatamente 5.000 imagens de cada classe.

*Referência*: Aprendendo várias camadas de recursos de imagens minúsculas, Alex Krizhevsky, 2009.

* Usaremos a versão disponível no tensorflow.


In [ ]:
# carregamos o dataset
(trainX, trainY), (testX, testY) = cifar10.load_data()

In [ ]:
print("Train samples:", trainX.shape, trainY.shape)
print("Test samples:", testX.shape, testY.shape)


Train samples: (50000, 32, 32, 3) (50000, 1)
Test samples: (10000, 32, 32, 3) (10000, 1)


In [ ]:
NUM_CLASSES = 10
cifar10_classes = ["airplane", "automobile", "bird", "cat", "deer",
                   "dog", "frog", "horse", "ship", "truck"]

###Preparação dos dados de entrada

**Reformatação**

Redefinimos o formato das imagens do dataset, tanto a base de treino como a base de teste, para o mínimo aceito pelo VGG16: 48x48x3.

**Normalização**

Normalizamos os dados de entrada com a função:
$$x_{norm} = \frac{x}{255}$$

Também transformamos de classe da variável alvo em um vetor de codificação one-hot.

In [ ]:
# Convertemos os valores dos pixels em float
trainX = trainX.astype('float32')
testX = testX.astype('float32')

#input_shape = (48, 48, 3)

# normalizamos para escala [0-1]
X_train = (trainX / 255)
X_test = (testX / 255)

# resize train set
#X_train = []
#for img in trainX:
 # X_train.append(np.resize(img, input_shape) / 255)

#X_train = np.array(X_train)
#print(X_train.shape)

# resize test set
#X_test = []
#for img in testX:
 # X_test.append(np.resize(img, input_shape) / 255)

#X_test = np.array(X_test)
#print(X_test.shape)

# convertemos os rótulos ou variável alvo
Y_train = to_categorical(trainY, NUM_CLASSES)
Y_test = to_categorical(testY, NUM_CLASSES)

## Construção do modelo



Aqui construíremos nosso modelo. Usaremos como base o modelo VGG16 pré-treinado com o conjunto de dados ImageNet.

A ideia do Transfer Learning é usar o modelo pré-treinado para extrair algumas características dos nossos dados. Essas extração de características foi previamente aprendida no ImageNet

Para que o modelo seja treinado para o contexto dos nossos dados, excluímos as últimas camadas totalmente conectadas do VGG16, adicionando novas camadas para serem treinadas. Assim, podemos pensar que estamos apenas usando a rede pré-treinada para extrair características e treinando uma rede mlp nova com essas características.

É importante que façamos com que os pesos das camadas extratoras da VGG16 não se alterem durante o treinamento. Apenas as camadas totalmente conectadas que adicionaremos ao modelo é que serão alteradas durante o treinamento.



Para treinamento do modelo, usaremos o algoritmo de otimização ADAM, com a função objetiva como a entropia cruzada categórica. Nossa principal métrica de desempenho será a acurácia.

In [ ]:
# Carregamos o modelo VGG16 sem as últimas camadas totalmente conectadas (include_top=False)
pre_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

newInput = Input(batch_shape=(None, 32, 32, 3))
resizedImg = Lambda(lambda image: tf.compat.v1.image.resize_images(image, (224, 224)))(newInput)
newOutputs = pre_model(resizedImg)
pre_model = Model(newInput, newOutputs)

# Aqui fazemos com que as camadas do modelo pré-treinado não sejam alteradas durante o treino
for layer in pre_model.layers:
  layer.trainable = False

# Criamos então um modelo sequential onde temos o VGG16 seguido das novas camadas conectadas
def define_model():
  model = Sequential()

  model.add(pre_model)

  model.add(Flatten())

  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.2))

  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))

  model.add(Dense(10, activation='softmax'))

  opt = Adam(learning_rate=0.001)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
# descrição do modelo
model = define_model()
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_1 (Functional)        (None, 7, 7, 512)         14714688  
                                                                 
 flatten_7 (Flatten)         (None, 25088)             0         
                                                                 
 dense_23 (Dense)            (None, 512)               12845568  
                                                                 
 dropout_21 (Dropout)        (None, 512)               0         
                                                                 
 dense_24 (Dense)            (None, 256)               131328    
                                                                 
 dropout_22 (Dropout)        (None, 256)               0         
                                                                 
 dense_25 (Dense)            (None, 10)               

In [ ]:
model.fit(X_train, Y_train, batch_size=32, epochs=10)

Epoch 1/10
1563/1563 [==============================] - 198s 126ms/step - loss: 1.3976 - accuracy: 0.5005
Epoch 2/10
1563/1563 [==============================] - 194s 124ms/step - loss: 1.1305 - accuracy: 0.5944
Epoch 3/10
1563/1563 [==============================] - 194s 124ms/step - loss: 1.0364 - accuracy: 0.6284
Epoch 4/10
1563/1563 [==============================] - 194s 124ms/step - loss: 0.9809 - accuracy: 0.6487
Epoch 5/10
1563/1563 [==============================] - 194s 124ms/step - loss: 0.9324 - accuracy: 0.6663
Epoch 6/10
1563/1563 [==============================] - 194s 124ms/step - loss: 0.9054 - accuracy: 0.6777
Epoch 7/10
1563/1563 [==============================] - 194s 124ms/step - loss: 0.8767 - accuracy: 0.6875
Epoch 8/10
1563/1563 [==============================] - 193s 124ms/step - loss: 0.8794 - accuracy: 0.6863
Epoch 9/10
1563/1563 [==============================] - 194s 124ms/step - loss: 0.8759 - accuracy: 0.6894
Epoch 10/10
1563/1563 [=======================

##Avaliação do modelo com o conjunto de teste

---



In [ ]:
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 39s 123ms/step - loss: 0.8319 - accuracy: 0.7151
Test loss: 0.831900954246521
Test accuracy: 0.7150999903678894


In [ ]:
# Carregamos o modelo VGG16 sem as últimas camadas totalmente conectadas (include_top=False)
model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

newInput = Input(batch_shape=(None, 32, 32, 3))
resizedImg = Lambda(lambda image: tf.compat.v1.image.resize_images(image, (48, 48)))(newInput)
newOutputs = model(resizedImg)
model = Model(newInput, newOutputs)

# Aqui fazemos com que as camadas do modelo pré-treinado não sejam alteradas durante o treino
for layer in pre_model.layers:
  layer.trainable = False

# Add the same dense layers as in VGG16
output = model.output
output = Flatten()(output)
output = Dense(units=512, activation='relu')(output)
output = Dense(units=256, activation='relu')(output)
output = Dense(units=10, activation='softmax')(output)
model = Model(model.input, output)


In [ ]:
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 lambda_11 (Lambda)          (None, 48, 48, 3)         0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_11 (Flatten)        (None, 512)               0         
                                                                 
 dense_35 (Dense)            (None, 512)               262656    
                                                                 
 dense_36 (Dense)            (None, 256)               131328    
                                                                 
 dense_37 (Dense)            (None, 10)                2570

In [ ]:
opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [ ]:
model.fit(X_train, Y_train, epochs=10, batch_size=32)

Epoch 1/10
1563/1563 [==============================] - 60s 38ms/step - loss: 1.7868 - accuracy: 0.2901
Epoch 2/10
1563/1563 [==============================] - 59s 38ms/step - loss: 1.3515 - accuracy: 0.4831
Epoch 3/10
1563/1563 [==============================] - 61s 39ms/step - loss: 1.0906 - accuracy: 0.6075
Epoch 4/10
1563/1563 [==============================] - 61s 39ms/step - loss: 0.9398 - accuracy: 0.6688
Epoch 5/10
1563/1563 [==============================] - 60s 39ms/step - loss: 0.8270 - accuracy: 0.7127
Epoch 6/10
1563/1563 [==============================] - 60s 38ms/step - loss: 0.7443 - accuracy: 0.7471
Epoch 7/10
1563/1563 [==============================] - 59s 38ms/step - loss: 0.6889 - accuracy: 0.7676
Epoch 8/10
1563/1563 [==============================] - 60s 38ms/step - loss: 0.6190 - accuracy: 0.7936
Epoch 9/10
1563/1563 [==============================] - 59s 38ms/step - loss: 0.5874 - accuracy: 0.8046
Epoch 10/10
1563/1563 [==============================] - 60s 38m

In [ ]:
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 4s 12ms/step - loss: 0.7489 - accuracy: 0.7661
Test loss: 0.7489198446273804
Test accuracy: 0.7660999894142151


In [ ]:
#!/usr/bin/env python3
"""
This script has the method
preprocess_data(X, Y):
"""
import tensorflow.keras as K


def preprocess_data(X, Y):
        """ This method has the preprocess to train a model """
        X = X.astype('float32')
        X_p = K.applications.vgg16.preprocess_input(X)
        Y_p = K.utils.to_categorical(Y, 10)
        return(X_p, Y_p)

if __name__ == "__main__":
    (Xt, Yt), (X, Y) = K.datasets.cifar10.load_data()
    X_p, Y_p = preprocess_data(Xt, Yt)
    Xv_p, Yv_p = preprocess_data(X, Y)
    base_model = K.applications.vgg16.VGG16(include_top=False,
                                            weights='imagenet',
                                            pooling='avg'
                                            )

    model= K.Sequential()
    model.add(K.layers.UpSampling2D())
    model.add(base_model)
    model.add(K.layers.Flatten())
    model.add(K.layers.Dense(512, activation=('relu')))
    model.add(K.layers.Dropout(0.2))
    model.add(K.layers.Dense(256, activation=('relu')))
    model.add(K.layers.Dropout(0.2))
    model.add(K.layers.Dense(10, activation=('softmax')))
    callback = []
    def decay(epoch):
        """ This method create the alpha"""
        return 0.001 / (1 + 1 * 30)
    callback += [K.callbacks.LearningRateScheduler(decay, verbose=1)]
    callback += [K.callbacks.ModelCheckpoint('cifar10.h5',
                                             save_best_only=True,
                                             mode='min'
                                             )]
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x=X_p, y=Y_p,
              batch_size=128,
              validation_data=(Xv_p, Yv_p),
              epochs=10, shuffle=True,
              callbacks=callback,
              verbose=1
              )


Epoch 1: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 1/10
391/391 [==============================] - 91s 201ms/step - loss: 1.3837 - accuracy: 0.5350 - val_loss: 0.6017 - val_accuracy: 0.7987 - lr: 3.2258e-05

Epoch 2: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 2/10
391/391 [==============================] - 77s 196ms/step - loss: 0.5351 - accuracy: 0.8261 - val_loss: 0.4378 - val_accuracy: 0.8580 - lr: 3.2258e-05

Epoch 3: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 3/10
391/391 [==============================] - 78s 200ms/step - loss: 0.3500 - accuracy: 0.8875 - val_loss: 0.3623 - val_accuracy: 0.8839 - lr: 3.2258e-05

Epoch 4: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 4/10
391/391 [==============================] - 80s 204ms/step - loss: 0.2309 - accuracy: 0.9241 - val_loss: 0.3567 - val_accuracy: 0.8896 - lr: 3.2258e-05

Epoch 5: LearningRateSchedu

In [ ]:
score = model.evaluate(X_p, Y_p)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

1563/1563 [==============================] - 23s 15ms/step - loss: 0.0242 - accuracy: 0.9924
Test loss: 0.024182196706533432
Test accuracy: 0.9923999905586243


In [ ]:
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

base_model.trainable = False

model = tf.keras.Sequential([
tf.keras.layers.InputLayer(input_shape=(32, 32, 3)),
tf.keras.layers.UpSampling2D(size = (2,2)),
tf.keras.layers.UpSampling2D(size = (2,2)),
tf.keras.layers.UpSampling2D(size = (2,2)),
base_model,
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dense(256, activation='relu'),
tf.keras.layers.Dropout(0.3),
tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dense(10, activation='softmax')])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 up_sampling2d_16 (UpSamplin  (None, 64, 64, 3)        0         
 g2D)                                                            
                                                                 
 up_sampling2d_17 (UpSamplin  (None, 128, 128, 3)      0         
 g2D)                                                            
                                                                 
 up_sampling2d_18 (UpSamplin  (None, 256, 256, 3)      0         
 g2D)                                                            
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten_17 (Flatten)        (None, 32768)             0         
                                                     

In [ ]:
opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=10, batch_size=32)

Epoch 1/10
1563/1563 [==============================] - 253s 158ms/step - loss: 1.3098 - accuracy: 0.5384
Epoch 2/10
1563/1563 [==============================] - 244s 156ms/step - loss: 0.9805 - accuracy: 0.6570
Epoch 3/10
1563/1563 [==============================] - 244s 156ms/step - loss: 0.8792 - accuracy: 0.6965
Epoch 4/10
1563/1563 [==============================] - 244s 156ms/step - loss: 0.8105 - accuracy: 0.7203
Epoch 5/10
1563/1563 [==============================] - 243s 156ms/step - loss: 0.7515 - accuracy: 0.7400
Epoch 6/10
1563/1563 [==============================] - 243s 156ms/step - loss: 0.7122 - accuracy: 0.7555
Epoch 7/10
1563/1563 [==============================] - 244s 156ms/step - loss: 0.6747 - accuracy: 0.7676
Epoch 8/10
1563/1563 [==============================] - 244s 156ms/step - loss: 0.6282 - accuracy: 0.7820
Epoch 9/10
1563/1563 [==============================] - 244s 156ms/step - loss: 0.6030 - accuracy: 0.7926
Epoch 10/10
1563/1563 [=======================

In [ ]:
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 49s 155ms/step - loss: 0.8484 - accuracy: 0.7257
Test loss: 0.8484376072883606
Test accuracy: 0.7257000207901001


In [ ]:
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

base_model.trainable = False

model = tf.keras.Sequential([
tf.keras.layers.InputLayer(input_shape=(32, 32, 3)),
tf.keras.layers.UpSampling2D(size = (2,2)),
tf.keras.layers.UpSampling2D(size = (2,2)),
tf.keras.layers.UpSampling2D(size = (2,2)),
base_model,
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dense(256, activation='relu'),
tf.keras.layers.Dropout(0.3),
tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dense(10, activation='softmax')])

callback = []
def decay(epoch):
        """ This method create the alpha"""
        return 0.001 / (1 + 1 * 30)
callback += [K.callbacks.LearningRateScheduler(decay, verbose=1)]
callback += [K.callbacks.ModelCheckpoint('cifar10.h5',
                                             save_best_only=True,
                                             mode='min'
                                             )]

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 up_sampling2d_19 (UpSamplin  (None, 64, 64, 3)        0         
 g2D)                                                            
                                                                 
 up_sampling2d_20 (UpSamplin  (None, 128, 128, 3)      0         
 g2D)                                                            
                                                                 
 up_sampling2d_21 (UpSamplin  (None, 256, 256, 3)      0         
 g2D)                                                            
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten_18 (Flatten)        (None, 32768)             0         
                                                     

In [ ]:
model.fit(X_train,
          Y_train,
          epochs=10, shuffle=True,
          callbacks=callback,
          batch_size=32)


Epoch 1: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 1/10
1563/1563 [==============================] - 248s 158ms/step - loss: 1.4224 - accuracy: 0.5029 - lr: 3.2258e-05

Epoch 2: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 2/10
1563/1563 [==============================] - 247s 158ms/step - loss: 1.0556 - accuracy: 0.6364 - lr: 3.2258e-05

Epoch 3: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 3/10
1563/1563 [==============================] - 247s 158ms/step - loss: 0.9323 - accuracy: 0.6776 - lr: 3.2258e-05

Epoch 4: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 4/10
1563/1563 [==============================] - 246s 158ms/step - loss: 0.8535 - accuracy: 0.7048 - lr: 3.2258e-05

Epoch 5: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 5/10
1563/1563 [==============================] - 246s 157ms/step - loss: 0.7971 - accuracy: 0.72

In [ ]:
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 49s 156ms/step - loss: 0.7788 - accuracy: 0.7319
Test loss: 0.7787634134292603
Test accuracy: 0.7318999767303467


In [ ]:
#!/usr/bin/env python3
"""
This script has the method
preprocess_data(X, Y):
"""
import tensorflow.keras as K


def preprocess_data(X, Y):
        """ This method has the preprocess to train a model """
        X = X.astype('float32')
        X_p = K.applications.vgg16.preprocess_input(X)
        Y_p = K.utils.to_categorical(Y, 10)
        return(X_p, Y_p)

if __name__ == "__main__":
    (Xt, Yt), (X, Y) = K.datasets.cifar10.load_data()
    assert Xt.shape == (50000, 32, 32, 3)
    assert X.shape == (10000, 32, 32, 3)
    assert Yt.shape == (50000, 1)
    assert Y.shape == (10000, 1)

    X_p, Y_p = preprocess_data(Xt, Yt)
    Xv_p, Yv_p = preprocess_data(X, Y)
    base_model = K.applications.vgg16.VGG16(include_top=False,
                                            weights='imagenet',
                                            pooling='avg'
                                            )

    model= K.Sequential()
    model.add(K.layers.UpSampling2D())
    model.add(base_model)
    model.add(K.layers.Flatten())
    model.add(K.layers.Dense(512, activation=('relu')))
    model.add(K.layers.Dropout(0.2))
    model.add(K.layers.Dense(256, activation=('relu')))
    model.add(K.layers.Dropout(0.2))
    model.add(K.layers.Dense(10, activation=('softmax')))
    callback = []
    def decay(epoch):
        """ This method create the alpha"""
        return 0.001 / (1 + 1 * 30)
    callback += [K.callbacks.LearningRateScheduler(decay, verbose=1)]
    callback += [K.callbacks.ModelCheckpoint('cifar10.h5',
                                             save_best_only=True,
                                             mode='min'
                                             )]
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x=X_p, y=Y_p,
              batch_size=128,
              validation_data=(Xv_p, Yv_p),
              epochs=10, shuffle=True,
              callbacks=callback,
              verbose=1
              )

    predicted_x = model.predict(Xv_p)
    residuals = np.argmax(predicted_x,1)!=np.argmax(Yv_p,1)

    loss = sum(residuals)/len(residuals)
    print("the validation 0/1 loss is: ",loss)


Epoch 1: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 1/10
391/391 [==============================] - 73s 169ms/step - loss: 1.4178 - accuracy: 0.5241 - val_loss: 0.6176 - val_accuracy: 0.7979 - lr: 3.2258e-05

Epoch 2: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 2/10
391/391 [==============================] - 63s 161ms/step - loss: 0.5275 - accuracy: 0.8298 - val_loss: 0.4005 - val_accuracy: 0.8703 - lr: 3.2258e-05

Epoch 3: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 3/10
391/391 [==============================] - 63s 161ms/step - loss: 0.3363 - accuracy: 0.8909 - val_loss: 0.3411 - val_accuracy: 0.8897 - lr: 3.2258e-05

Epoch 4: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 4/10
391/391 [==============================] - 63s 160ms/step - loss: 0.2267 - accuracy: 0.9254 - val_loss: 0.3145 - val_accuracy: 0.9021 - lr: 3.2258e-05

Epoch 5: LearningRateSchedu

In [ ]:
score = model.evaluate(Xv_p, Yv_p)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 5s 15ms/step - loss: 0.4336 - accuracy: 0.9047
Test loss: 0.43364930152893066
Test accuracy: 0.904699981212616
